In [1]:
API_KEY = '...'
BASE_URL = "https://api.anthropic.com/v1/"
MODEL = 'claude-3-7-sonnet-20250219'

SYSTEM_PROMPT = "You are OpenManus, an all-capable AI assistant, aimed at solving any task presented by the user. You have various tools at your disposal that you can call upon to efficiently complete complex requests. Whether it's programming, information retrieval, file processing, or web browsing, you can handle it all."

NEXT_STEP_PROMPT = """You can interact with the computer using PythonExecute, save important content and information files through FileSaver, open browsers with BrowserUseTool, and retrieve information using GoogleSearch.

PythonExecute: Execute Python code to interact with the computer system, data processing, automation tasks, etc.

FileSaver: Save files locally, such as txt, py, html, etc.

BrowserUseTool: Open, browse, and use web browsers.If you open a local HTML file, you must provide the absolute path to the file.

GoogleSearch: Perform web information retrieval

Terminate: End the current interaction when the task is complete or when you need additional information from the user. Use this tool to signal that you've finished addressing the user's request or need clarification before proceeding further.

Based on user needs, proactively select the most appropriate tool or combination of tools. For complex tasks, you can break down the problem and use different tools step by step to solve it. After using each tool, clearly explain the execution results and suggest the next steps.

Always maintain a helpful, informative tone throughout the interaction. If you encounter any limitations or need more details, clearly communicate this to the user before terminating.
"""

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "python_execute",
            "description": "Executes Python code string. Note: Only print outputs are visible, function return values are not captured. Use print statements to see results.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "The Python code to execute.",
                    }
                },
                "required": ["code"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "file_saver",
            "description": "Save content to a local file at a specified path.\nUse this tool when you need to save text, code, or generated content to a file on the local filesystem.\nThe tool accepts content and a file path, and saves the content to that location.\n",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "(required) The content to save to the file.",
                    },
                    "file_path": {
                        "type": "string",
                        "description": "(required) The path where the file should be saved, including filename and extension.",
                    },
                    "mode": {
                        "type": "string",
                        "description": "(optional) The file opening mode. Default is 'w' for write. Use 'a' for append.",
                        "enum": ["w", "a"],
                        "default": "w",
                    },
                },
                "required": ["content", "file_path"],
            },
        },
    },
]

In [2]:
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall
import json
from openai import AsyncOpenAI
from openai.types.chat import ChatCompletionMessage

def python_execute(code):
    import threading
    result = {"observation": ""}

    def run_code():
        try:
            import sys
            from io import StringIO

            output_buffer = StringIO()
            sys.stdout = output_buffer

            exec(code, {}, {})

            sys.stdout = sys.__stdout__

            result["observation"] = output_buffer.getvalue()

        except Exception as e:
            result["observation"] = str(e)
            result["success"] = False

    thread = threading.Thread(target=run_code)
    thread.start()
    thread.join()
    return result

def file_saver(content: str, file_path: str, mode: str = "w") -> str:
    """
    Save content to a file at the specified path.

    Args:
        content (str): The content to save to the file.
        file_path (str): The path where the file should be saved.
        mode (str, optional): The file opening mode. Default is 'w' for write. Use 'a' for append.

    Returns:
        str: A message indicating the result of the operation.
    """
    import os
    try:
        # Ensure the directory exists
        directory = os.path.dirname(file_path)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        # Write directly to the file
        with open(file_path, mode, encoding="utf-8") as file:
            file.write(content)

        return f"Content successfully saved to {file_path}"
    except Exception as e:
        return f"Error saving file: {str(e)}"

def make_func_call(command: ChatCompletionMessageToolCall):
    kwargs = json.loads(command.function.arguments or "{}")
    
    name_to_function = {
        "python_execute": python_execute,
        "file_saver": file_saver,
    }
    result = name_to_function[command.function.name](**kwargs)
    observation = (
                    f"Observed output of cmd `{command.function.name}` executed:\n{str(result)}"
                    if result
                    else f"Cmd `{name}` completed with no output"
                )
    return {
        'role': 'tool',
        'content': observation,
        'tool_call_id': command.id,
        'name': command.function.name,
    }

In [3]:
# Initialize the AsyncOpenAI client
client = AsyncOpenAI(
    api_key=API_KEY, 
    base_url=BASE_URL
)

In [4]:
messages_queue = [    {
        'role': 'system',
        'content': SYSTEM_PROMPT,
    },
]

messages_queue.append({
        'role': 'user',
        'content': 'write a python function with two args, a string and a char, so that the function return how many char are there in the string',
    }
)

messages_queue.append({        
        'role': 'user',
        'content': NEXT_STEP_PROMPT,
    })

In [5]:
async def ask_gpt(messages: list[dict[str, str]]) -> ChatCompletionMessage:
    response = await client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=None,
        tools=TOOLS,
        tool_choice='auto'
    )
    return response.choices[0].message

In [6]:
async def think(messages_queue):
    response = await ask_gpt(messages_queue)
    if not response.tool_calls:
        messages_queue.append({
            'role': 'assistant',
            'content': response.content,
        })
        return response
        
    
    tool_call = response.tool_calls[0]
    messages_queue.append({
        'role': 'assistant',
        'content': response.content,
        'tool_calls': [{'id': tool_call.id,
       'type': 'function',
       'function': {'name': tool_call.function.name,
        'arguments': tool_call.function.arguments}
         }
        ]
    })
    
    return response

async def act(message_queue, response):
    messages_queue.append(make_func_call(response.tool_calls[0]))

In [7]:
# expect to use python_execution tool
response = await think(messages_queue)
assert len(response.tool_calls) == 1
await act(messages_queue, response)

In [8]:
messages_queue

[{'role': 'system',
  'content': "You are OpenManus, an all-capable AI assistant, aimed at solving any task presented by the user. You have various tools at your disposal that you can call upon to efficiently complete complex requests. Whether it's programming, information retrieval, file processing, or web browsing, you can handle it all."},
 {'role': 'user',
  'content': 'write a python function with two args, a string and a char, so that the function return how many char are there in the string'},
 {'role': 'user',
  'content': "You can interact with the computer using PythonExecute, save important content and information files through FileSaver, open browsers with BrowserUseTool, and retrieve information using GoogleSearch.\n\nPythonExecute: Execute Python code to interact with the computer system, data processing, automation tasks, etc.\n\nFileSaver: Save files locally, such as txt, py, html, etc.\n\nBrowserUseTool: Open, browse, and use web browsers.If you open a local HTML file,

In [9]:
messages_queue.append({
    'role': 'user',
    'content': \
    "save the previously generated python function including test cases into my current dir with name foo.py"
})

In [10]:
# expect to use file_saver tool
response = await think(messages_queue)
assert len(response.tool_calls) == 1
await act(messages_queue, response)

In [11]:
messages_queue

[{'role': 'system',
  'content': "You are OpenManus, an all-capable AI assistant, aimed at solving any task presented by the user. You have various tools at your disposal that you can call upon to efficiently complete complex requests. Whether it's programming, information retrieval, file processing, or web browsing, you can handle it all."},
 {'role': 'user',
  'content': 'write a python function with two args, a string and a char, so that the function return how many char are there in the string'},
 {'role': 'user',
  'content': "You can interact with the computer using PythonExecute, save important content and information files through FileSaver, open browsers with BrowserUseTool, and retrieve information using GoogleSearch.\n\nPythonExecute: Execute Python code to interact with the computer system, data processing, automation tasks, etc.\n\nFileSaver: Save files locally, such as txt, py, html, etc.\n\nBrowserUseTool: Open, browse, and use web browsers.If you open a local HTML file,